In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def relu6(x):
    return layers.ReLU(max_value=6.0)(x)

def inverted_residual(x, out_ch, stride, expand_ratio):

    in_ch = int(x.shape[-1]) # 224,224,3

    # 1×1 expansion
    if expand_ratio != 1:
        y = layers.Conv2D(filters = in_ch * expand_ratio, kernel_size=1, padding="same", use_bias=False)(x)
        y = layers.BatchNormalization()(y)
        y = relu6(y)
    else:
        y = x

    # 3×3 depthwise
    y = layers.DepthwiseConv2D(kernel_size=3, strides=stride, padding="same", use_bias=False)(y)
    y = layers.BatchNormalization()(y)
    y = relu6(y)

    # 1×1 projection
    y = layers.Conv2D(filters = out_ch, kernel_size=1, padding="same",  use_bias=False)(y)
    y = layers.BatchNormalization()(y)

    # Inverted residual (residual connection)
    if stride == 1 and in_ch == out_ch:
        y = layers.Add()([x, y])
    return y

def MobileNetV2(input_shape=(224, 224, 3), classes=1000, alpha=1.0):
    
    def channel(ch):
        return int(ch * alpha)

    inp = layers.Input(shape=input_shape)

    x = layers.Conv2D(filters=channel(32), kernel_size=3, strides=2, padding="same", use_bias=False)(inp)
    x = layers.BatchNormalization()(x)
    x = relu6(x)

    x = inverted_residual(x, channel(16), stride=1, expand_ratio=1)

    x = inverted_residual(x, channel(24), stride=2, expand_ratio=6)
    x = inverted_residual(x, channel(24), stride=1, expand_ratio=6)

    x = inverted_residual(x, channel(32), stride=2, expand_ratio=6)
    x = inverted_residual(x, channel(32), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(32), stride=1, expand_ratio=6)

    x = inverted_residual(x, channel(64), stride=2, expand_ratio=6)
    x = inverted_residual(x, channel(64), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(64), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(64), stride=1, expand_ratio=6)

    x = inverted_residual(x, channel(96), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(96), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(96), stride=1, expand_ratio=6)

    x = inverted_residual(x, channel(160), stride=2, expand_ratio=6)
    x = inverted_residual(x, channel(160), stride=1, expand_ratio=6)
    x = inverted_residual(x, channel(160), stride=1, expand_ratio=6)

    x = inverted_residual(x, channel(320), stride=1, expand_ratio=6)

    x = layers.Conv2D(filters=1280, kernel_size=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = relu6(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(classes, activation="softmax")(x)

    return models.Model(inp, out)

In [3]:
model = MobileNetV2(input_shape=(224,224,3), classes=1000, alpha=1)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 112, 112, 32)      │             128 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu (ReLU)                  │ (None, 112, 112, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d              │ (None, 112, 112, 32)      │             288 │ re_lu[0][0]                │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 112, 112, 32)      │             128 │ depthwise_conv2d[0][0]     │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_1 (ReLU)                │ (None, 112, 112, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 112, 112, 16)      │             512 │ re_lu_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 112, 112, 16)      │              64 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 112, 112, 96)      │           1,536 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 112, 112, 96)      │             384 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_2 (ReLU)                │ (None, 112, 112, 96)      │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_1            │ (None, 56, 56, 96)        │             864 │ re_lu_2[0][0]              │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 56, 56, 96)        │             384 │ depthwise_conv2d_1[0][0]   │
│ (BatchNormalization)          │                           │               

 Total params: 3,538,984 (13.50 MB)

 Trainable params: 3,504,872 (13.37 MB)

 Non-trainable params: 34,112 (133.25 KB)